In [3]:
# Installer les bibliothèques nécessaires
!pip install pydub SpeechRecognition pandas transformers openpyxl
!pip install transformers torchaudio

In [4]:
#Importation des bibliothèques nécessaires

import os
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, pipeline
from pydub import AudioSegment
from google.colab import drive

# Monter Google Drive

drive.mount('/content/drive')

# Fonction pour les fichiers audios

def read_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    return audio

# Charger le modèle et le tokenizer de Hugging Face

model_name = "facebook/wav2vec2-large-960h"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Fonction pour convertir l'audio en texte

def audio_to_text(file_path):

    # Convertir le fichier audio en WAV si ce n'est pas déjà

    if not file_path.endswith('.wav'):
        audio_segment = AudioSegment.from_file(file_path)
        wav_path = file_path.rsplit('.', 1)[0] + '.wav'
        audio_segment.export(wav_path, format='wav')
        file_path = wav_path

    # Charger l'audio et effectuer la reconnaissance vocale

    waveform, sample_rate = torchaudio.load(file_path)
    input_values = tokenizer(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values

    # Effectuer la prédiction

    with torch.no_grad():
        logits = model(input_values).logits

    # Obtenir les indices des tokens

    predicted_ids = logits.argmax(dim=-1)

    # Convertir les indices en texte

    text = tokenizer.batch_decode(predicted_ids)[0]
    return text

# Fonction pour analyser le sentiment du texte

def analyze_sentiment(text):
    sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    sentiment = sentiment_pipeline(text)
    return sentiment[0]['label'], sentiment[0]['score']

# Dossier contenant les fichiers audio

audio_folder = '/content/drive/My Drive/dataset_audio'
results = []

# Lire les fichiers audio dans le dossier

for index, filename in enumerate(os.listdir(audio_folder)):
    if filename.endswith('.wav') or filename.endswith('.mp3'):
        file_path = os.path.join(audio_folder, filename)
        try:
            text = audio_to_text(file_path)
            sentiment_label, sentiment_score = analyze_sentiment(text)

            results.append({
                'Numéro': index + 1,
                'Texte': text,
                'Sentiment': sentiment_label,
                'Score': sentiment_score
            })
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {filename}: {e}")

# Créer un DataFrame et l'exporter en Excel

df = pd.DataFrame(results)
df.to_excel('/content/drive/My Drive/resultats_analyse_sentiment.xlsx', index=False)

# Affichier le resultat

print(df)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


   Numéro                                              Texte Sentiment  \
0       1  AATOSO N T     ONTO MO  OSH MO O A HONA M  PA ...    1 star   
1       2  NOTN A  HASTON PCOMONSHOPOTONNCHOO OIS POO TO ...    1 star   
2       3  PMAPO AN  PUL  P AUUN PU  PU V  PPU  A  A  O  ...    1 star   
3       4  U OM TUOTEO MPU MO TEIMOO L P CONCANP ONNY HOM...    1 star   
4       5  OM TOOSOM  POO PO OM H FOTONO ONO PO O OMO P O...   3 stars   

      Score  
0  0.416714  
1  0.425537  
2  0.296365  
3  0.389075  
4  0.319892  
